In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, SymLogNorm, DivergingNorm
import matplotlib.colors as colors
plt.rcParams['figure.max_open_warning'] = 500
import qcodes as qc
import qcodes.instrument_drivers.nplab_drivers as npd
from scipy.optimize import curve_fit
from scipy.interpolate import interp2d, interp1d
from os import path
import colorcet as cc   # install quickly with "conda install colorcet"
from scipy.interpolate import RectBivariateSpline
from matplotlib import ticker
from scipy.ndimage import gaussian_filter
import matplotlib as mpl
from lmfit import Model
import glob

In [15]:
%matplotlib widget

In [3]:
cd ..

/Users/robertpolski/Documents/NPGroupResearch/Measurements/Triton/qcodes_data


In [4]:
figfile = 'Figures/W5_WALpeaks/'

In [5]:
qc.DataSet.default_io.base_location = path.abspath('.')

In [144]:
dlf1 = qc.load_data('data/2020-03-25/#003_W5_865_40-39_830_39-38_smallfieldsweep_-1Vbg_baseT_1nA_05-06-40')
dlf0 = qc.load_data('data/2020-03-24/#018_W5_865_40-39_830_39-38_fieldsweep_-1Vbg_baseT_1nA_17-13-50')

In [145]:
plt.figure()
plt.plot(dlf0.triton_field[:], gaussian_filter(dlf0.lockin865_X[:]/1e-9, 5))
plt.plot(dlf0.triton_field[:], gaussian_filter(dlf0.lockin830_X[:]/1e-9, 5))
plt.plot(dlf1.triton_field[:], gaussian_filter(dlf1.lockin865_X[:]/1e-9, 10))
plt.plot(dlf1.triton_field[:], gaussian_filter(dlf1.lockin830_X[:]/1e-9, 5))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# An attempt at WAL fitting at -6Vg

Note that the 0.80 degree device is on lockin830 (contacts 39-38)

In [146]:
V0 = 0.34
Vgatefull = 5.1
nu = np.interp(-6, [V0 - 7/4*Vgatefull, V0 + 7/4*Vgatefull], [-7, 7])  # the filling of -6 Vg
print(nu)

-4.972549019607843


In [147]:
nfull = 1.504322970117718
print(nu*nfull/4)  # the density of electrons, relative to CNP
print(nu*nfull/4 + nfull)  # the density of electrons, relative to full filling

-1.8700799275581046
-0.3657569574403865


In [148]:
dlf2 = qc.load_data('data/2020-03-24/#021_W5_865_40-39_830_39-38_fieldsweep_-6Vbg_baseT_1nA_23-47-37')
dlast = np.where(np.isnan(dlf2.triton_field[:]))[0][0]
b = dlf2.triton_field[:dlast]
R = dlf2.lockin830_X[:dlast]/1e-9
G = 1/R

In [149]:
bvalsn = np.linspace(-0.4, 0, 40*3+1)
bvalsp = np.linspace(0, 0.4, 40*3+1)
oneside = 0.5*(np.interp(bvalsn, b, G)[::-1] + np.interp(bvalsp, b, G))

bs = np.concatenate((bvalsn, bvalsp))
Gs = np.concatenate((oneside[::-1], oneside))

In [150]:
esqh = 1.602e-19**2/6.626e-34
G0 = oneside[0]

In [151]:
plt.figure()
# plt.plot(dlf2.triton_field[:], 1/gaussian_filter(dlf2.lockin865_X[:]/1e-9, 3))
plt.plot(bs, (gaussian_filter(Gs, 2))/esqh, '.r')
plt.xlabel('B (T)')
plt.ylabel('$\sigma(B)$ ($e^2/h$)')
plt.savefig('lowfieldpeak_raw', dpi=300, bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
from scipy.special import digamma

In [28]:
def F(x):
    return np.log(x) + digamma(0.5 + 1/x)

def dsigma(B):
    D = 100
    taubinv = 4*D*1.602e-19*B/(6.626e-34/(2*np.pi))
    tauphi = 5e-14
    tauasy = 1e-13
    tauso = 0.6e-15
    return -1*1.602e-19**2/(np.pi*6.626e-34)*(F(taubinv/tauphi**-1) - F(taubinv/(tauphi**-1 + 2*tauasy**-1)) - 2*F(taubinv/(tauphi**-1 + tauso**-1)))

In [29]:
plt.figure()
x = np.linspace(0.000001, 0.01, 501)
xs = np.concatenate((-1*x[::-1], x))
ys = np.concatenate((dsigma(x)[::-1], dsigma(x)))
plt.plot(xs*1000, ys/esqh)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Fitting attempts

In [152]:
def F(x):
    return np.log(x) + digamma(0.5 + 1/x)

def dsigma_fit(B, D, tauphi, tauasy, tauso):  # approximately 5e-5, 1e-15, 1e-19 (need to be in that size order but not sure of the details)
#     D = 1e-9
    taubinv = 4*D*1.602e-19*B/(6.626e-34/(2*np.pi))
    return -1*1.602e-19**2/(np.pi*6.626e-34)*(F(taubinv/tauphi**-1) - F(taubinv/(tauphi**-1 + 2*tauasy**-1)) - 2*F(taubinv/(tauphi**-1 + tauso**-1)))

WALmodel=Model(dsigma_fit)

print('parameter names: {}'.format(WALmodel.param_names))
print('independent variables: {}'.format(WALmodel.independent_vars))

parameter names: ['D', 'tauphi', 'tauasy', 'tauso']
independent variables: ['B']


Trying with stopping at each point

In [153]:
dstop = qc.load_data('data/2020-03-25/#009_W5_865_40-39_830_39-38_fieldsweep_-6Vbg_baseT_10nA_17-15-59')

In [154]:
len(dstop.lockin830_X)

201

In [155]:
plt.figure()  # it took 54 minutes to do this one sweep with 201 pts.
plt.plot(dstop.triton_field_set_stable_set, (1/(dstop.lockin830_X[:]/10e-9) - G0)/esqh, '.')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Other gate points

In [156]:
dlf3 = qc.load_data('data/2020-03-24/#019_W5_865_40-39_830_39-38_fieldsweep_-2Vbg_baseT_1nA_19-25-00')

In [157]:
plt.figure()
plt.plot(dlf3.triton_field[:], gaussian_filter(dlf3.lockin865_X[:]/1e-9, 3))
plt.plot(dlf3.triton_field[:], gaussian_filter(dlf3.lockin830_X[:]/1e-9, 3))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [158]:
dlf4 = qc.load_data('data/2020-03-24/#017_W5_865_40-39_830_39-38_fieldsweep_2Vbg_baseT_1nA_15-02-38')

In [159]:
plt.figure()
plt.plot(dlf4.triton_field[:], gaussian_filter(dlf4.lockin865_X[:]/1e-9, 3))
plt.plot(dlf4.triton_field[:], gaussian_filter(dlf4.lockin830_X[:]/1e-9, 3))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [160]:
dlf5 = qc.load_data('data/2020-03-24/#020_W5_865_40-39_830_39-38_fieldsweep_-3Vbg_baseT_1nA_21-36-10')

In [161]:
plt.figure()
plt.plot(dlf5.triton_field[:], 1/gaussian_filter(dlf5.lockin865_X[:]/1e-9, 3))
plt.plot(dlf5.triton_field[:], 1/gaussian_filter(dlf5.lockin830_X[:]/1e-9, 3))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Attempting low fields, an ensemble, near -6 to -7Vg

In [162]:
filenames0 = sorted(glob.glob('data/2020-03-25/#*/'))[-5:]
filenames1 = sorted(glob.glob('data/2020-03-26/#*/'))[:36]

In [163]:
# make one for each lockin (just look at X since the resistance is low)
dN = 501
f0rows = len(filenames0)
rows = len(filenames0) + len(filenames1)
totarray830 = np.zeros((rows, dN))
totarray865 = np.zeros((rows, dN))
bpts = np.linspace(-0.05, 0.05, dN)
gpts = np.linspace(-6, -6.8, 41)

d0 = qc.load_data(filenames0[0][:-1])
dlast = np.where(np.isnan(d0.triton_field[:]))[0][0]
d830_0 = np.interp(bpts, d0.triton_field[:dlast], d0.lockin830_X[:dlast])
d865_0 = np.interp(bpts, d0.triton_field[:dlast], d0.lockin865_X[:dlast])

for i, f in enumerate(filenames0):
    data = qc.load_data(f[:-1])
    dlast = np.where(np.isnan(data.triton_field[:]))[0][0]
    totarray830[i, :] = np.interp(bpts, data.triton_field[:dlast], data.lockin830_X[:dlast]) - np.average(data.lockin830_X[:dlast]) + np.average(d830_0)
    totarray865[i, :] = np.interp(bpts, data.triton_field[:dlast], data.lockin865_X[:dlast]) - np.average(data.lockin865_X[:dlast]) + np.average(d865_0)
    
for i, f in enumerate(filenames1):
    data = qc.load_data(f[:-1])
    dlast = np.where(np.isnan(data.triton_field[:]))[0][0]
    totarray830[f0rows + i, :] = np.interp(bpts, data.triton_field[:dlast], data.lockin830_X[:dlast]) - np.average(data.lockin830_X[:dlast]) + np.average(d830_0)
    totarray865[f0rows + i, :] = np.interp(bpts, data.triton_field[:dlast], data.lockin865_X[:dlast]) - np.average(data.lockin865_X[:dlast]) + np.average(d865_0)

In [164]:
0.1/400

0.00025

In [165]:
plt.figure()  # electrodes 39-38
plt.pcolormesh(bpts*1000, gpts, 1/(totarray830/10e-9)/esqh, cmap=cc.cm.linear_kryw_0_100_c71)
plt.colorbar(label='$\sigma$ ($e^2/h$)')
plt.xlabel('B (mT)')
plt.ylabel('V$_g$')
# plt.savefig(figfile + 'WAL_2Dplot_3K', dpi=300, bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'V$_g$')

In [166]:
# np.savetxt('AnalysisNotebooks/W5bsweepsmall39-38_10nA_3K.txt', totarray830, fmt='%.9e', header='#array of lockin830 voltage readings (divide by 10e-9 to get Rxx) at 3K for electrodes 39-38, with axis 0 being backgate voltage from -6 (top) to -6.8 (bottom), every 0.02 V (41 pts)  and field from -50 to 50 mT, every 0.2 mT (501 pts)')

In [167]:
# average
plt.figure()
plt.plot(bpts*1000, 1/np.average(totarray830[10:, :]/10e-9, axis=0)/esqh)
# plt.plot(bpts, 1/(totarray830[20, :]/10e-9)/esqh)
# plt.plot(bpts, 1/np.average(totarray865/10e-9, axis=0)/esqh)
# plt.plot(bpts, 1/np.average(totarray865[:5, :]/10e-9, axis=0)/esqh)
plt.axis([-30, 30, 70, 74])
plt.xlabel('B (mT)')
plt.ylabel('$\sigma$ ($e^2/h$)')
# plt.savefig(figfile + 'ensembleaverage_dispersive_3K_2_zoom', dpi=300, bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, '$\\sigma$ ($e^2/h$)')

In [168]:
plt.figure()  # electrodes 40-39
plt.pcolormesh(bpts, gpts, totarray865/10e-9, cmap=cc.cm.linear_kryw_0_100_c71)
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [169]:
# np.savetxt('AnalysisNotebooks/W5bsweepsmall40-39_10nA_3K.txt', totarray830, fmt='%.9e', header='#array of lockin830 voltage readings (divide by 10e-9 to get Rxx) at 3K for electrodes 39-38, with axis 0 being backgate voltage from -6 (top) to -6.8 (bottom), every 0.02 V (41 pts)  and field from -50 to 50 mT, every 0.2 mT (501 pts)')

In [170]:
plt.figure()
plt.plot(bpts, 1/np.average(totarray865/10e-9, axis=0))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [171]:
plt.figure()
# plt.plot(dlf2.triton_field[:], 1/gaussian_filter(dlf2.lockin865_X[:]/1e-9, 3))
# plt.plot(bs, (gaussian_filter(Gs, 2) - G0)/esqh, '.r')
plt.plot(bpts, (1/np.average(totarray830[1:2, :]/10e-9, axis=0) - G0)/esqh, '.b')
plt.plot(b, (gaussian_filter(G, 2) - G0)/esqh, '.k')
plt.plot(dstop.triton_field_set_stable_set, (1/(dstop.lockin830_X[:]/10e-9) - G0)/esqh, '.r')
plt.xlabel('B (T)')
plt.ylabel('$\sigma(B) - \sigma(0)$ ($e^2/h$)')
# plt.axis((-0.5, 0.5, -40, 5))
# plt.axis(a)
# plt.savefig('lowfieldpeak_withoriginal', dpi=300, bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, '$\\sigma(B) - \\sigma(0)$ ($e^2/h$)')

# Now at -2 Vg

In [51]:
V0 = 0.34
Vgatefull = 5.1
nu = np.interp(-2, [V0 - 7/4*Vgatefull, V0 + 7/4*Vgatefull], [-7, 7])  # the filling of -6 Vg
print(nu)
nfull = 1.504322970117718
print(nu*nfull/4)  # the density of electrons, relative to CNP in 10^12 /cm^2

-1.8352941176470585
-0.6902187745245999


## Base Temperature

First a single point

In [52]:
dn2 = qc.load_data('data/2020-03-26/#038_W5_865_40-39_830_39-38_smallfieldsweep_-2Vbg_baseT_10nA_13-15-03')
dn2last = np.where(np.isnan(dn2.triton_field))[0][0]
bn2 = dn2.triton_field[:dn2last]

In [53]:
plt.figure()
plt.plot(dn2.triton_field[:]*1000, 1/(dn2.lockin865_X[:]/10e-9)/esqh)
plt.axis((-20, 20, 38, 39.2))
plt.ylabel('$\sigma (B)$ ($e^2/h$)')
plt.xlabel('B (mT)')
plt.savefig(figfile + 'W5_40-39_-2Vbg.png', dpi=300, bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [54]:
# Now do the same averaging with the latest data from different gate points.
files_n2 = sorted(glob.glob('data/2020-03-26/#*/'))[53:58]

In [55]:
files_n2  # just the files where the data range is -50 to 50 mT

['data/2020-03-26/#054_W5_865_40-39_830_39-38_baseT_WALfieldsw_Vbg-1.74_10nA_20-57-58/',
 'data/2020-03-26/#055_W5_865_40-39_830_39-38_baseT_WALfieldsw_Vbg-1.76_10nA_21-19-35/',
 'data/2020-03-26/#056_W5_865_40-39_830_39-38_baseT_WALfieldsw_Vbg-1.78_10nA_21-41-15/',
 'data/2020-03-26/#057_W5_865_40-39_830_39-38_baseT_WALfieldsw_Vbg-1.80_10nA_22-02-54/',
 'data/2020-03-26/#058_W5_865_40-39_830_39-38_baseT_WALfieldsw_Vbg-1.82_10nA_22-24-33/']

In [56]:
sorted(glob.glob('data/2020-03-26/#*/'))[58:]

['data/2020-03-26/#059_W5_865_40-39_830_39-38_baseT_WALfieldsw_Vbg-1.84_10nA_22-48-21/',
 'data/2020-03-26/#060_W5_865_40-39_830_39-38_baseT_WALfieldsw_Vbg-1.86_10nA_22-59-27/',
 'data/2020-03-26/#061_W5_865_40-39_830_39-38_baseT_WALfieldsw_Vbg-1.88_10nA_23-10-33/',
 'data/2020-03-26/#062_W5_865_40-39_830_39-38_baseT_WALfieldsw_Vbg-1.90_10nA_23-21-41/',
 'data/2020-03-26/#063_W5_865_40-39_830_39-38_baseT_WALfieldsw_Vbg-1.92_10nA_23-32-49/',
 'data/2020-03-26/#064_W5_865_40-39_830_39-38_baseT_WALfieldsw_Vbg-1.94_10nA_23-43-56/',
 'data/2020-03-26/#065_W5_865_40-39_830_39-38_baseT_WALfieldsw_Vbg-1.96_10nA_23-55-01/']

In [57]:
files_n2_1 = sorted(glob.glob('data/2020-03-26/#*/'))[58:65]  # at slightly smaller field range of -/+25 mT

In [58]:
files_n2_1

['data/2020-03-26/#059_W5_865_40-39_830_39-38_baseT_WALfieldsw_Vbg-1.84_10nA_22-48-21/',
 'data/2020-03-26/#060_W5_865_40-39_830_39-38_baseT_WALfieldsw_Vbg-1.86_10nA_22-59-27/',
 'data/2020-03-26/#061_W5_865_40-39_830_39-38_baseT_WALfieldsw_Vbg-1.88_10nA_23-10-33/',
 'data/2020-03-26/#062_W5_865_40-39_830_39-38_baseT_WALfieldsw_Vbg-1.90_10nA_23-21-41/',
 'data/2020-03-26/#063_W5_865_40-39_830_39-38_baseT_WALfieldsw_Vbg-1.92_10nA_23-32-49/',
 'data/2020-03-26/#064_W5_865_40-39_830_39-38_baseT_WALfieldsw_Vbg-1.94_10nA_23-43-56/',
 'data/2020-03-26/#065_W5_865_40-39_830_39-38_baseT_WALfieldsw_Vbg-1.96_10nA_23-55-01/']

In [59]:
files_n2_2 = sorted(glob.glob('data/2020-03-27/#*/'))[:19]
files_n2_2

['data/2020-03-27/#001_W5_865_40-39_830_39-38_baseT_WALfieldsw_Vbg-1.98_10nA_00-06-06/',
 'data/2020-03-27/#002_W5_865_40-39_830_39-38_baseT_WALfieldsw_Vbg-2.00_10nA_00-17-12/',
 'data/2020-03-27/#003_W5_865_40-39_830_39-38_baseT_WALfieldsw_Vbg-2.02_10nA_00-28-18/',
 'data/2020-03-27/#004_W5_865_40-39_830_39-38_baseT_WALfieldsw_Vbg-2.04_10nA_00-39-22/',
 'data/2020-03-27/#005_W5_865_40-39_830_39-38_baseT_WALfieldsw_Vbg-2.06_10nA_00-50-29/',
 'data/2020-03-27/#006_W5_865_40-39_830_39-38_baseT_WALfieldsw_Vbg-2.08_10nA_01-01-35/',
 'data/2020-03-27/#007_W5_865_40-39_830_39-38_baseT_WALfieldsw_Vbg-2.10_10nA_01-12-42/',
 'data/2020-03-27/#008_W5_865_40-39_830_39-38_baseT_WALfieldsw_Vbg-2.12_10nA_01-23-48/',
 'data/2020-03-27/#009_W5_865_40-39_830_39-38_baseT_WALfieldsw_Vbg-2.14_10nA_01-34-56/',
 'data/2020-03-27/#010_W5_865_40-39_830_39-38_baseT_WALfieldsw_Vbg-2.16_10nA_01-46-00/',
 'data/2020-03-27/#011_W5_865_40-39_830_39-38_baseT_WALfieldsw_Vbg-2.18_10nA_01-57-08/',
 'data/2020-03-27/#01

In [60]:
d0 = qc.load_data(files_n2[0][:-1])
dlast = np.where(np.isnan(d0.triton_field))[0][0]
len(d0.triton_field[:dlast])  # Use 501 pts for the range. Now only use -/+ 25 mT, 251 pts

517

In [61]:
d1 = qc.load_data(files_n2_1[0][:-1])
dlast = np.where(np.isnan(d1.triton_field))[0][0]
dlast   # so use 251 interpolated points, since the field sweep rate is the same

259

In [62]:
nB_0 = 501
barr_0 = np.linspace(-0.05, 0.05, nB_0)  # we will trim this down later when we combine arrays.
garr_0 = np.array([-1.74, -1.76, -1.78, -1.8, -1.82])  # add -2 into the front and uncomment the lines below if you want it to +/-50mT
ng_0 = len(garr_0)
arrayn2_830 = np.zeros((ng_0, nB_0))
arrayn2_865 = np.zeros((ng_0, nB_0))

# arrayn2_830[0, :] = np.interp(barr_0, bn2, dn2.lockin830_X[:dn2last])
# arrayn2_865[0, :] = np.interp(barr_0, bn2, dn2.lockin865_X[:dn2last])

for i, f in enumerate(files_n2):
    data = qc.load_data(f[:-1])
    dlast = np.where(np.isnan(data.triton_field))[0][0]
    b = data.triton_field[:dlast]
    arrayn2_830[i, :] = np.interp(barr_0, b, data.lockin830_X[:dlast])
    arrayn2_865[i, :] = np.interp(barr_0, b, data.lockin865_X[:dlast])


In [63]:
nB = 251
barr = np.linspace(-0.025, 0.025, nB)  # we will trim this down later when we combine arrays.
garr_1 = np.array([-1.84, -1.86, -1.88, -1.90, -1.92, -1.94, -1.96])
ng_1 = len(garr_1)
garr_2 = np.linspace(-1.98, -2.34, 19) #np.array([-1.98, -2.0, -2.02, -2.04, -2.06])
ng_2 = len(garr_2)
arrayn2_830_1 = np.zeros((ng_1 + ng_2, nB))
arrayn2_865_1 = np.zeros((ng_1 + ng_2, nB))

for i, f in enumerate(files_n2_1):
    data = qc.load_data(f[:-1])
    dlast = np.where(np.isnan(data.triton_field))[0][0]
    b = data.triton_field[:dlast]
    arrayn2_830_1[i, :] = np.interp(barr, b, data.lockin830_X[:dlast])
    arrayn2_865_1[i, :] = np.interp(barr, b, data.lockin865_X[:dlast])
    
for i, f in enumerate(files_n2_2):
    data = qc.load_data(f[:-1])
    dlast = np.where(np.isnan(data.triton_field))[0][0]
    b = data.triton_field[:dlast]
    arrayn2_830_1[ng_1 + i, :] = np.interp(barr, b, data.lockin830_X[:dlast])
    arrayn2_865_1[ng_1 + i, :] = np.interp(barr, b, data.lockin865_X[:dlast])

In [64]:
totgs = np.concatenate((garr_0, garr_1, garr_2))
totarrayn2_830 = np.vstack((arrayn2_830[:, 125:376], arrayn2_830_1))
totarrayn2_865 = np.vstack((arrayn2_865[:, 125:376], arrayn2_865_1))

In [65]:
totarrayn2_865.shape

(31, 251)

In [139]:
# Note that index 11 (Vg = -1.96) is bad
np.savetxt('AnalysisNotebooks/W5_WAL_arrays/W5_10nA_-2Vbg_baseT_40-39.txt', totarrayn2_865, fmt='%.8e', header='#array of lockin865 voltage readings (divide by 10e-9 to get Rxx) at base Temp for electrodes 40-39, with axis 0 being backgate voltage from -1.74 (top) to -2.34 (bottom)--here note that index 11 (-1.96 Vbg) was stopped partway so should be excluded-- every 0.02 V (31 pts) and field from -25 to 25 mT, every 0.2 mT (251 pts)')
np.savetxt('AnalysisNotebooks/W5_WAL_arrays/W5_10nA_-2Vbg_baseT_39-38.txt', totarrayn2_830, fmt='%.8e', header='#array of lockin830 voltage readings (divide by 10e-9 to get Rxx) at base Temp for electrodes 39-38, with axis 0 being backgate voltage from -1.74 (top) to -2.34 (bottom)--here note that index 11 (-1.96 Vbg) was stopped partway so should be excluded-- every 0.02 V (31 pts) and field from -25 to 25 mT, every 0.2 mT (251 pts)')

In [66]:
totgs

array([-1.74, -1.76, -1.78, -1.8 , -1.82, -1.84, -1.86, -1.88, -1.9 ,
       -1.92, -1.94, -1.96, -1.98, -2.  , -2.02, -2.04, -2.06, -2.08,
       -2.1 , -2.12, -2.14, -2.16, -2.18, -2.2 , -2.22, -2.24, -2.26,
       -2.28, -2.3 , -2.32, -2.34])

In [132]:
plt.figure()
for i in range(12):
    G830 = 1/(totarrayn2_830[i, :]/10e-9)
    plt.plot(barr*1000, G830/esqh, label='{} V'.format(totgs[i]))
    
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [68]:
plt.figure()
plt.plot(barr*1000, 1/np.average(totarrayn2_830[1:, :]/10e-9, axis=0)/esqh, '.k')  # skip the -2Vg point
# plt.axis((-20, 20, 38.2, 38.8))
# plt.savefig(figfile + 'W5_39-38near-2WALpeak_avgfewpts.png', dpi=300, bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [131]:
plt.figure()  # index 11 (-1.96V) must have been stopped or something like that.
for i in range(10,17):
    G865 = 1/(totarrayn2_865[i, :]/10e-9)
    plt.plot(barr*1000, G865/esqh, label='{} V'.format(totgs[i]), lw=0.3)
    
plt.legend(loc='upper right')
# plt.axis(a)
plt.ylabel('$\sigma (B)$ ($e^2/h$)')
plt.xlabel('B (mT)')

# plt.savefig(figfile + 'W5_near-2WALpeak_fewpts.png', dpi=300, bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'B (mT)')

In [70]:
a = plt.axis()

In [71]:
def symmetrize(b, G, bpeak):
    """Enter the field values in b, conductance values in G, and the best estimate of the 
    field (in T) at the peak that should be at 0 field.
    
    This function assumes that bpeak is slightly less than 0 and, more importantly,
    that there are more field points above than below this point. It just concatenates points
    that can't be symmetrized.
    
    Returns a symmetrized b, G, G0"""
    
    zptind = npd.val_to_index([bpeak], b)[0]
    G0 = G[zptind]
    Gpsym = 0.5*(G[:zptind+1][::-1] + G[zptind:zptind + zptind + 1])
    Gsymm = np.concatenate((Gpsym[::-1], Gpsym[1:]))

    bpsymm = (b[zptind:2*zptind + 1] - b[zptind])*1000
    bsymm = np.concatenate((-1*bpsymm[::-1], bpsymm[1:]))
    
    return bsymm, Gsymm, G0

In [72]:
totarrayn2_865.shape

(31, 251)

In [73]:
totgs

array([-1.74, -1.76, -1.78, -1.8 , -1.82, -1.84, -1.86, -1.88, -1.9 ,
       -1.92, -1.94, -1.96, -1.98, -2.  , -2.02, -2.04, -2.06, -2.08,
       -2.1 , -2.12, -2.14, -2.16, -2.18, -2.2 , -2.22, -2.24, -2.26,
       -2.28, -2.3 , -2.32, -2.34])

In [74]:
totarrayn2_865.shape

(31, 251)

In [75]:
plt.figure()

bsymm, Gsymm, G0 = symmetrize(barr, 1/np.average(totarrayn2_865[:, :]/10e-9, axis=0)/esqh, -1.19/1000)

plt.plot(bsymm, Gsymm - G0, '.r')
# plt.plot(barr*1000, 1/np.average(totarrayn2_865/10e-9, axis=0)/esqh - G0, '.k') # raw
# plt.axis((-6, 6, -0.2, 0.05))
plt.ylabel('$\sigma (B) - \sigma (0)$ ($e^2/h$)')
plt.xlabel('B (mT)')
# plt.savefig(figfile + 'W5_near-2WALpeak_avgfewmorepts_39-38_-6to6mT.png', dpi=300, bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

-0.00 is not a value in the array
Plotted -0.00 instead


Text(0.5, 0, 'B (mT)')

In [76]:
# attempt at manual fitting
plt.figure()

bsymm, Gsymm, G0 = symmetrize(barr, 1/np.average(totarrayn2_865/10e-9, axis=0)/esqh, -1.19/1000)

plt.plot(bsymm, Gsymm - G0, '.r')
plt.axis((-6, 6, -0.2, 0.05))
plt.ylabel('$\sigma (B) - \sigma (0)$ ($e^2/h$)')
plt.xlabel('B (mT)')

def dsigma(B):
    D = 3.2  # not sure if these values are anywhere close to the right range.
    taubinv = 4*D*1.602e-19*B/(6.626e-34/(2*np.pi))
    tauphi = 35e-14
    tauasy = 18e-14
    tauso = 80e-15
    return -1*1.602e-19**2/(np.pi*6.626e-34)*(F(taubinv/tauphi**-1) - F(taubinv/(tauphi**-1 + 2*tauasy**-1)) - 2*F(taubinv/(tauphi**-1 + tauso**-1)))

x = np.linspace(0.000001, 0.01, 501)
xs = np.concatenate((-1*x[::-1], x))
ys = np.concatenate((dsigma(x)[::-1], dsigma(x)))
plt.plot(xs*1000, ys/esqh)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

-0.00 is not a value in the array
Plotted -0.00 instead


In [77]:
dGn2_865 = np.zeros(totarrayn2_865.shape)
pind = npd.val_to_index([-1.19/1000], barr)[0]
for i in range(totarrayn2_865.shape[0]):
    dGn2_865[i, :] = 1/(totarrayn2_865[i, :]/10e-9)/esqh - 1/(totarrayn2_865[i, pind]/10e-9)/esqh

plt.figure()  # lockin865 (40-39)
# plt.pcolormesh(barr*1000, totgs, gaussian_filter((1/(totarrayn2_865/10e-9)/esqh - G0), (1, 1)))
plt.pcolormesh(barr*1000, totgs, gaussian_filter(dGn2_865, (0.8,0.8)), vmin=-1, vmax=0.25)
plt.colorbar()
plt.ylabel('V$_{g}$')
plt.xlabel('B (mT)')
# plt.savefig(figfile + 'W5_near-2WALpeak_2Dmap_40-39.png', dpi=300, bbox_inches='tight')

-0.00 is not a value in the array
Plotted -0.00 instead


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'B (mT)')

In [78]:
dGn2_830 = np.zeros(totarrayn2_830.shape)
pind = npd.val_to_index([-1.19/1000], barr)[0]
for i in range(totarrayn2_830.shape[0]):
    dGn2_830[i, :] = 1/(totarrayn2_830[i, :]/10e-9)/esqh - 1/(totarrayn2_830[i, pind]/10e-9)/esqh

plt.figure()  # lockin830 (39-38)
# plt.pcolormesh(barr*1000, totgs, gaussian_filter((1/(totarrayn2_830/10e-9)/esqh - G0), (1, 1)))
plt.pcolormesh(barr*1000, totgs, gaussian_filter(dGn2_830, (0.8, 0.8)))
plt.colorbar()
plt.ylabel('V$_{g}$')
plt.xlabel('B (mT)')
# plt.savefig(figfile + 'W5_near-2WALpeak_2Dmap_39-38.png', dpi=300, bbox_inches='tight')

-0.00 is not a value in the array
Plotted -0.00 instead


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'B (mT)')

## 300 mK

In [97]:
files_n2_300 = sorted(glob.glob('data/2020-03-28/#*/'))[17:47]
files_n2_300

['data/2020-03-28/#018_W5_865_40-39_830_39-38_300mK_WALfieldsw_Vbg-1.74_10nA_14-12-09/',
 'data/2020-03-28/#019_W5_865_40-39_830_39-38_300mK_WALfieldsw_Vbg-1.76_10nA_14-23-12/',
 'data/2020-03-28/#020_W5_865_40-39_830_39-38_300mK_WALfieldsw_Vbg-1.78_10nA_14-34-17/',
 'data/2020-03-28/#021_W5_865_40-39_830_39-38_300mK_WALfieldsw_Vbg-1.80_10nA_14-45-17/',
 'data/2020-03-28/#022_W5_865_40-39_830_39-38_300mK_WALfieldsw_Vbg-1.82_10nA_14-56-22/',
 'data/2020-03-28/#023_W5_865_40-39_830_39-38_300mK_WALfieldsw_Vbg-1.84_10nA_15-07-27/',
 'data/2020-03-28/#024_W5_865_40-39_830_39-38_300mK_WALfieldsw_Vbg-1.86_10nA_15-18-32/',
 'data/2020-03-28/#025_W5_865_40-39_830_39-38_300mK_WALfieldsw_Vbg-1.88_10nA_15-29-37/',
 'data/2020-03-28/#026_W5_865_40-39_830_39-38_300mK_WALfieldsw_Vbg-1.90_10nA_15-40-42/',
 'data/2020-03-28/#027_W5_865_40-39_830_39-38_300mK_WALfieldsw_Vbg-1.92_10nA_15-51-47/',
 'data/2020-03-28/#028_W5_865_40-39_830_39-38_300mK_WALfieldsw_Vbg-1.94_10nA_16-02-51/',
 'data/2020-03-28/#02

In [99]:
nB = 251
barr = np.linspace(-0.025, 0.025, nB)
garr2_300 = np.linspace(-1.74, -2.32, 30)
ng2_300 = len(garr2_300)
arrayn2_830_300 = np.zeros((ng2_300, nB))
arrayn2_865_300 = np.zeros((ng2_300, nB))

for i, f in enumerate(files_n2_300):
    data = qc.load_data(f[:-1])
    dlast = np.where(np.isnan(data.triton_field))[0][0]
    b = data.triton_field[:dlast]
    arrayn2_830_300[i, :] = np.interp(barr, b, data.lockin830_X[:dlast])
    arrayn2_865_300[i, :] = np.interp(barr, b, data.lockin865_X[:dlast])

In [141]:
np.savetxt('AnalysisNotebooks/W5_WAL_arrays/W5_10nA_-2Vbg_300mK_40-39.txt', arrayn2_865_300, fmt='%.8e', header='#array of lockin865 voltage readings (divide by 10e-9 to get Rxx) at 300mK for electrodes 40-39, with axis 0 being backgate voltage from -1.74 (top) to -2.32 (bottom) every 0.02 V (30 pts) and field from -25 to 25 mT, every 0.2 mT (251 pts)')
np.savetxt('AnalysisNotebooks/W5_WAL_arrays/W5_10nA_-2Vbg_300mK_39-38.txt', arrayn2_830_300, fmt='%.8e', header='#array of lockin830 voltage readings (divide by 10e-9 to get Rxx) at 300mK for electrodes 39-38, with axis 0 being backgate voltage from -1.74 (top) to -2.32 (bottom) every 0.02 V (30 pts) and field from -25 to 25 mT, every 0.2 mT (251 pts)')

In [108]:
dGn2_865_300 = np.zeros(arrayn2_865_300.shape)
pind = npd.val_to_index([-1.19/1000], barr)[0]
for i in range(arrayn2_865_300.shape[0]):
    dGn2_865_300[i, :] = 1/(arrayn2_865_300[i, :]/10e-9)/esqh - 1/(arrayn2_865_300[i, pind]/10e-9)/esqh

plt.figure()  # lockin830 (40-39)
plt.pcolormesh(barr*1000, garr2_300, gaussian_filter(dGn2_865_300, (0.8, 0.8)))
plt.colorbar()
plt.ylabel('V$_{g}$')
plt.xlabel('B (mT)')
# plt.savefig(figfile + 'W5_near-2WALpeak_2Dmap_40-39_300mK.png', dpi=300, bbox_inches='tight')

-0.00 is not a value in the array
Plotted -0.00 instead


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'B (mT)')

In [107]:
dGn2_830_300 = np.zeros(arrayn2_830_300.shape)
pind = npd.val_to_index([-1.19/1000], barr)[0]
for i in range(arrayn2_830_300.shape[0]):
    dGn2_830_300[i, :] = 1/(arrayn2_830_300[i, :]/10e-9)/esqh - 1/(arrayn2_830_300[i, pind]/10e-9)/esqh

plt.figure()  # lockin830 (39-38)
plt.pcolormesh(barr*1000, garr2_300, gaussian_filter(dGn2_830_300, (0.8, 0.8)))
plt.colorbar()
plt.ylabel('V$_{g}$')
plt.xlabel('B (mT)')
# plt.savefig(figfile + 'W5_near-2WALpeak_2Dmap_39-38_300mK.png', dpi=300, bbox_inches='tight')

-0.00 is not a value in the array
Plotted -0.00 instead


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'B (mT)')

## 500 mK

In [125]:
files_n2_500 = sorted(glob.glob('data/2020-03-29/#*/'))[28:58]

In [126]:
nB = 251
barr = np.linspace(-0.025, 0.025, nB)
garr2_500 = np.linspace(-1.74, -2.32, 30)
ng2_500 = len(garr2_300)
arrayn2_830_500 = np.zeros((ng2_500, nB))
arrayn2_865_500 = np.zeros((ng2_500, nB))

for i, f in enumerate(files_n2_500):
    data = qc.load_data(f[:-1])
    dlast = np.where(np.isnan(data.triton_field))[0][0]
    b = data.triton_field[:dlast]
    arrayn2_830_500[i, :] = np.interp(barr, b, data.lockin830_X[:dlast])
    arrayn2_865_500[i, :] = np.interp(barr, b, data.lockin865_X[:dlast])

In [142]:
np.savetxt('AnalysisNotebooks/W5_WAL_arrays/W5_10nA_-2Vbg_500mK_40-39.txt', arrayn2_865_500, fmt='%.8e', header='#array of lockin865 voltage readings (divide by 10e-9 to get Rxx) at 500mK for electrodes 40-39, with axis 0 being backgate voltage from -1.74 (top) to -2.32 (bottom) every 0.02 V (30 pts) and field from -25 to 25 mT, every 0.2 mT (251 pts)')
np.savetxt('AnalysisNotebooks/W5_WAL_arrays/W5_10nA_-2Vbg_500mK_39-38.txt', arrayn2_830_500, fmt='%.8e', header='#array of lockin830 voltage readings (divide by 10e-9 to get Rxx) at 500mK for electrodes 39-38, with axis 0 being backgate voltage from -1.74 (top) to -2.32 (bottom) every 0.02 V (30 pts) and field from -25 to 25 mT, every 0.2 mT (251 pts)')

In [127]:
dGn2_865_500 = np.zeros(arrayn2_865_500.shape)
pind = npd.val_to_index([-1.19/1000], barr)[0]
for i in range(arrayn2_865_500.shape[0]):
    dGn2_865_500[i, :] = 1/(arrayn2_865_500[i, :]/10e-9)/esqh - 1/(arrayn2_865_500[i, pind]/10e-9)/esqh

plt.figure()  # lockin830 (40-39)
plt.pcolormesh(barr*1000, garr2_500, gaussian_filter(dGn2_865_500, (0.8, 0.8)))
plt.colorbar()
plt.ylabel('V$_{g}$')
plt.xlabel('B (mT)')
# plt.savefig(figfile + 'W5_near-2WALpeak_2Dmap_40-39_500mK.png', dpi=300, bbox_inches='tight')

-0.00 is not a value in the array
Plotted -0.00 instead


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'B (mT)')

In [128]:
dGn2_830_500 = np.zeros(arrayn2_830_500.shape)
pind = npd.val_to_index([-1.19/1000], barr)[0]
for i in range(arrayn2_830_500.shape[0]):
    dGn2_830_500[i, :] = 1/(arrayn2_830_500[i, :]/10e-9)/esqh - 1/(arrayn2_830_500[i, pind]/10e-9)/esqh

plt.figure()  # lockin830 (39-38)
plt.pcolormesh(barr*1000, garr2_500, gaussian_filter(dGn2_830_500, (0.8, 0.8)))
plt.colorbar()
plt.ylabel('V$_{g}$')
plt.xlabel('B (mT)')
# plt.savefig(figfile + 'W5_near-2WALpeak_2Dmap_39-38_500mK.png', dpi=300, bbox_inches='tight')

-0.00 is not a value in the array
Plotted -0.00 instead


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'B (mT)')

## 700 mK

In [229]:
files_n2_700 = sorted(glob.glob('data/2020-03-29/#*/'))[-12:] + sorted(glob.glob('data/2020-03-30/#*/'))[:18]

In [231]:
garr2_700 = np.linspace(-1.74, -2.32, 30)
ng2_700 = len(garr2_700)
arrayn2_830_700 = np.zeros((ng2_700, nB))
arrayn2_865_700 = np.zeros((ng2_700, nB))

for i, f in enumerate(files_n2_700):
    data = qc.load_data(f[:-1])
    dlast = np.where(np.isnan(data.triton_field))[0][0]
    b = data.triton_field[:dlast]
    arrayn2_830_700[i, :] = np.interp(barr, b, data.lockin830_X[:dlast])
    arrayn2_865_700[i, :] = np.interp(barr, b, data.lockin865_X[:dlast])

In [232]:
np.savetxt('AnalysisNotebooks/W5_WAL_arrays/W5_10nA_-2Vbg_700mK_40-39.txt', arrayn2_865_700, fmt='%.8e', header='#array of lockin865 voltage readings (divide by 10e-9 to get Rxx) at 700mK for electrodes 40-39, with axis 0 being backgate voltage from -1.74 (top) to -2.32 (bottom) every 0.02 V (30 pts) and field from -25 to 25 mT, every 0.2 mT (251 pts)')
np.savetxt('AnalysisNotebooks/W5_WAL_arrays/W5_10nA_-2Vbg_700mK_39-38.txt', arrayn2_830_700, fmt='%.8e', header='#array of lockin830 voltage readings (divide by 10e-9 to get Rxx) at 700mK for electrodes 39-38, with axis 0 being backgate voltage from -1.74 (top) to -2.32 (bottom) every 0.02 V (30 pts) and field from -25 to 25 mT, every 0.2 mT (251 pts)')

## 900 mK

In [94]:
files_n2_900 = sorted(glob.glob('data/2020-03-27/#*/'))[97:] + sorted(glob.glob('data/2020-03-28/#*/'))[:17]
files_n2_900

['data/2020-03-27/#098_W5_865_40-39_830_39-38_0p9K_WALfieldsw_Vbg-1.74_10nA_19-19-23/',
 'data/2020-03-27/#099_W5_865_40-39_830_39-38_0p9K_WALfieldsw_Vbg-1.76_10nA_19-30-31/',
 'data/2020-03-27/#100_W5_865_40-39_830_39-38_0p9K_WALfieldsw_Vbg-1.78_10nA_19-41-39/',
 'data/2020-03-27/#101_W5_865_40-39_830_39-38_0p9K_WALfieldsw_Vbg-1.80_10nA_19-52-46/',
 'data/2020-03-27/#102_W5_865_40-39_830_39-38_0p9K_WALfieldsw_Vbg-1.82_10nA_20-03-53/',
 'data/2020-03-27/#103_W5_865_40-39_830_39-38_0p9K_WALfieldsw_Vbg-1.84_10nA_20-14-57/',
 'data/2020-03-27/#104_W5_865_40-39_830_39-38_0p9K_WALfieldsw_Vbg-1.86_10nA_20-26-01/',
 'data/2020-03-27/#105_W5_865_40-39_830_39-38_0p9K_WALfieldsw_Vbg-1.88_10nA_20-37-05/',
 'data/2020-03-27/#106_W5_865_40-39_830_39-38_0p9K_WALfieldsw_Vbg-1.90_10nA_20-48-13/',
 'data/2020-03-27/#107_W5_865_40-39_830_39-38_0p9K_WALfieldsw_Vbg-1.92_10nA_20-59-21/',
 'data/2020-03-27/#108_W5_865_40-39_830_39-38_0p9K_WALfieldsw_Vbg-1.94_10nA_21-10-25/',
 'data/2020-03-27/#109_W5_865_40

In [95]:
del files_n2_900[13]  # get rid of the stuck magnet (magnet likely restarted and randomly read -50 mT)

In [110]:
nB = 251
barr = np.linspace(-0.025, 0.025, nB)
garr2_900 = np.linspace(-1.74, -2.32, 30)
ng2_900 = len(garr2_300)
arrayn2_830_900 = np.zeros((ng2_900, nB))
arrayn2_865_900 = np.zeros((ng2_900, nB))

for i, f in enumerate(files_n2_900):
    data = qc.load_data(f[:-1])
    dlast = np.where(np.isnan(data.triton_field))[0][0]
    b = data.triton_field[:dlast]
    arrayn2_830_900[i, :] = np.interp(barr, b, data.lockin830_X[:dlast])
    arrayn2_865_900[i, :] = np.interp(barr, b, data.lockin865_X[:dlast])

In [143]:
np.savetxt('AnalysisNotebooks/W5_WAL_arrays/W5_10nA_-2Vbg_900mK_40-39.txt', arrayn2_865_900, fmt='%.8e', header='#array of lockin865 voltage readings (divide by 10e-9 to get Rxx) at 900mK for electrodes 40-39, with axis 0 being backgate voltage from -1.74 (top) to -2.32 (bottom) every 0.02 V (30 pts) and field from -25 to 25 mT, every 0.2 mT (251 pts)')
np.savetxt('AnalysisNotebooks/W5_WAL_arrays/W5_10nA_-2Vbg_900mK_39-38.txt', arrayn2_830_900, fmt='%.8e', header='#array of lockin830 voltage readings (divide by 10e-9 to get Rxx) at 900mK for electrodes 39-38, with axis 0 being backgate voltage from -1.74 (top) to -2.32 (bottom) every 0.02 V (30 pts) and field from -25 to 25 mT, every 0.2 mT (251 pts)')

In [111]:
dGn2_865_900 = np.zeros(arrayn2_865_900.shape)
pind = npd.val_to_index([-1.19/1000], barr)[0]
for i in range(arrayn2_865_900.shape[0]):
    dGn2_865_900[i, :] = 1/(arrayn2_865_900[i, :]/10e-9)/esqh - 1/(arrayn2_865_900[i, pind]/10e-9)/esqh

plt.figure()  # lockin830 (40-39)
plt.pcolormesh(barr*1000, garr2_900, gaussian_filter(dGn2_865_900, (0.8, 0.8)))
plt.colorbar()
plt.ylabel('V$_{g}$')
plt.xlabel('B (mT)')
# plt.savefig(figfile + 'W5_near-2WALpeak_2Dmap_40-39_900mK.png', dpi=300, bbox_inches='tight')

-0.00 is not a value in the array
Plotted -0.00 instead


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'B (mT)')

In [112]:
dGn2_830_900 = np.zeros(arrayn2_830_900.shape)
pind = npd.val_to_index([-1.19/1000], barr)[0]
for i in range(arrayn2_830_900.shape[0]):
    dGn2_830_900[i, :] = 1/(arrayn2_830_900[i, :]/10e-9)/esqh - 1/(arrayn2_830_900[i, pind]/10e-9)/esqh

plt.figure()  # lockin830 (39-38)
plt.pcolormesh(barr*1000, garr2_900, gaussian_filter(dGn2_830_900, (0.8, 0.8)))
plt.colorbar()
plt.ylabel('V$_{g}$')
plt.xlabel('B (mT)')
# plt.savefig(figfile + 'W5_near-2WALpeak_2Dmap_39-38_900mK.png', dpi=300, bbox_inches='tight')

-0.00 is not a value in the array
Plotted -0.00 instead


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'B (mT)')

In [287]:
files_n2_3k = sorted(glob.glob('data/2020-03-31/#*/'))[2:32]

In [288]:
nB = 251
barr = np.linspace(-0.025, 0.025, nB)
garr2_3k = np.linspace(-1.74, -2.32, 30)
ng2_3k = len(garr2_300)
arrayn2_830_3k = np.zeros((ng2_3k, nB))
arrayn2_865_3k = np.zeros((ng2_3k, nB))

for i, f in enumerate(files_n2_3k):
    data = qc.load_data(f[:-1])
    dlast = np.where(np.isnan(data.triton_field))[0][0]
    b = data.triton_field[:dlast]
    arrayn2_830_3k[i, :] = np.interp(barr, b, data.lockin830_X[:dlast])
    arrayn2_865_3k[i, :] = np.interp(barr, b, data.lockin865_X[:dlast])

In [289]:
np.savetxt('AnalysisNotebooks/W5_WAL_arrays/W5_10nA_-2Vbg_3K_40-39.txt', arrayn2_865_3k, fmt='%.8e', header='#array of lockin865 voltage readings (divide by 10e-9 to get Rxx) at 3K for electrodes 40-39, with axis 0 being backgate voltage from -1.74 (top) to -2.32 (bottom) every 0.02 V (30 pts) and field from -25 to 25 mT, every 0.2 mT (251 pts)')
np.savetxt('AnalysisNotebooks/W5_WAL_arrays/W5_10nA_-2Vbg_3K_39-38.txt', arrayn2_830_3k, fmt='%.8e', header='#array of lockin830 voltage readings (divide by 10e-9 to get Rxx) at 3K for electrodes 39-38, with axis 0 being backgate voltage from -1.74 (top) to -2.32 (bottom) every 0.02 V (30 pts) and field from -25 to 25 mT, every 0.2 mT (251 pts)')

## Combined

In [ ]:
# Lockin865 (40-39)
plt.figure()
plt.plot()

#  Near -3.5 Vg

Base temp

In [172]:
files_n35_base = sorted(glob.glob('data/2020-03-27/#*/'))[45:71]
files_n35_base

['data/2020-03-27/#046_W5_865_40-39_830_39-38_baseT_WALfieldsw_Vbg-3.26_10nA_08-26-17/',
 'data/2020-03-27/#047_W5_865_40-39_830_39-38_baseT_WALfieldsw_Vbg-3.28_10nA_08-37-21/',
 'data/2020-03-27/#048_W5_865_40-39_830_39-38_baseT_WALfieldsw_Vbg-3.30_10nA_08-48-26/',
 'data/2020-03-27/#049_W5_865_40-39_830_39-38_baseT_WALfieldsw_Vbg-3.32_10nA_08-59-33/',
 'data/2020-03-27/#050_W5_865_40-39_830_39-38_baseT_WALfieldsw_Vbg-3.34_10nA_09-10-37/',
 'data/2020-03-27/#051_W5_865_40-39_830_39-38_baseT_WALfieldsw_Vbg-3.36_10nA_09-21-41/',
 'data/2020-03-27/#052_W5_865_40-39_830_39-38_baseT_WALfieldsw_Vbg-3.38_10nA_09-32-48/',
 'data/2020-03-27/#053_W5_865_40-39_830_39-38_baseT_WALfieldsw_Vbg-3.40_10nA_09-43-53/',
 'data/2020-03-27/#054_W5_865_40-39_830_39-38_baseT_WALfieldsw_Vbg-3.42_10nA_09-54-57/',
 'data/2020-03-27/#055_W5_865_40-39_830_39-38_baseT_WALfieldsw_Vbg-3.44_10nA_10-06-03/',
 'data/2020-03-27/#056_W5_865_40-39_830_39-38_baseT_WALfieldsw_Vbg-3.46_10nA_10-17-10/',
 'data/2020-03-27/#05

In [173]:
nB = 251
barr = np.linspace(-0.025, 0.025, nB)
garr_n35 = np.linspace(-3.26, -3.76, 26)
ng_n35 = len(garr_n35)
arrayn35_830_base = np.zeros((ng_n35, nB))
arrayn35_865_base = np.zeros((ng_n35, nB))

for i, f in enumerate(files_n35_base):
    data = qc.load_data(f[:-1])
    dlast = np.where(np.isnan(data.triton_field))[0][0]
    b = data.triton_field[:dlast]
    arrayn35_830_base[i, :] = np.interp(barr, b, data.lockin830_X[:dlast])
    arrayn35_865_base[i, :] = np.interp(barr, b, data.lockin865_X[:dlast])

In [174]:
np.savetxt('AnalysisNotebooks/W5_WAL_arrays/W5_10nA_-3p5Vbg_baseT_40-39.txt', arrayn35_865_base, fmt='%.8e', header='#array of lockin865 voltage readings (divide by 10e-9 to get Rxx) at base Temp for electrodes 40-39, with axis 0 being backgate voltage from -3.26 (top) to -3.76 (bottom) every 0.02 V (26 pts) and field from -25 to 25 mT, every 0.2 mT (251 pts)')
np.savetxt('AnalysisNotebooks/W5_WAL_arrays/W5_10nA_-3p5Vbg_baseT_39-38.txt', arrayn35_830_base, fmt='%.8e', header='#array of lockin830 voltage readings (divide by 10e-9 to get Rxx) at base Temp for electrodes 39-38, with axis 0 being backgate voltage from -3.26 (top) to -3.76 (bottom) every 0.02 V (26 pts) and field from -25 to 25 mT, every 0.2 mT (251 pts)')

In [866]:
plt.figure()  # lockin865 (40-39), -3.26 to -3.76, averaging 26 pts
plt.plot(barr*1000, 1/np.average(arrayn35_865/10e-9, axis=0)/esqh - 24.176, '.r')
plt.axis(a)
plt.xlabel('B (mT)')
plt.ylabel('$\sigma(B) - \sigma(0)$ ($e^2/h$)')
# plt.savefig(figfile + 'W5_near-3p5WALpeak_avg26pts_40-39_raw.png', dpi=300, bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [944]:
dGn35_865_base = np.zeros(arrayn35_865_base.shape)
pind = npd.val_to_index([-1.19/1000], barr)[0]
for i in range(arrayn35_865_base.shape[0]):
    dGn35_865_base[i, :] = 1/(arrayn35_865_base[i, :]/10e-9)/esqh - 1/(arrayn35_865_base[i, pind]/10e-9)/esqh

plt.figure()  # lockin865 (40-39)
# plt.pcolormesh(barr*1000, garr_n35, gaussian_filter((1/(arrayn35_865/10e-9)/esqh - 24.176), (1, 1)))
plt.pcolormesh(barr*1000, garr_n35, gaussian_filter(dGn35_865_base, (0.8, 0.8)))
plt.colorbar()
plt.ylabel('V$_{g}$')
plt.xlabel('B (mT)')
# plt.savefig(figfile + 'W5_near-3p5WALpeak_2d_40-39_raw.png', dpi=300, bbox_inches='tight')

-0.00 is not a value in the array
Plotted -0.00 instead


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'B (mT)')

In [872]:
a = plt.axis()

In [874]:

plt.figure()  # lockin830 (39-38), -3.26 to -3.76, averaging 26 pts
plt.plot(barr*1000, 1/np.average(arrayn35_830/10e-9, axis=0)/esqh - 29.5, '.r')
plt.axis(a)
plt.xlabel('B (mT)')
plt.ylabel('$\sigma(B) - \sigma(0)$ ($e^2/h$)')
# plt.savefig(figfile + 'W5_near-3p5WALpeak_avg26pts_39-38_raw.png', dpi=300, bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [945]:
dGn35_830 = np.zeros(arrayn35_830.shape)
pind = npd.val_to_index([-1.19/1000], barr)[0]
for i in range(arrayn35_830.shape[0]):
    dGn35_830[i, :] = 1/(arrayn35_830[i, :]/10e-9)/esqh - 1/(arrayn35_830[i, pind]/10e-9)/esqh

plt.figure()  # lockin830 (39-38)
# plt.pcolormesh(barr*1000, garr_n35, gaussian_filter((1/(arrayn35_830/10e-9)/esqh - 29.5), (1, 1)))
plt.pcolormesh(barr*1000, garr_n35, gaussian_filter(dGn35_830, (0.8, 0.8)))
plt.colorbar()
plt.ylabel('V$_{g}$')
plt.xlabel('B (mT)')
# plt.savefig(figfile + 'W5_near-3p5WALpeak_2d_39-38_raw.png', dpi=300, bbox_inches='tight')

-0.00 is not a value in the array
Plotted -0.00 instead


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'B (mT)')

300 mK

In [175]:
files_n35_300 = sorted(glob.glob('data/2020-03-28/#*/'))[47:] + sorted(glob.glob('data/2020-03-29/#*/'))[:2]

In [176]:
arrayn35_830_300 = np.zeros((ng_n35, nB))
arrayn35_865_300 = np.zeros((ng_n35, nB))

for i, f in enumerate(files_n35_300):
    data = qc.load_data(f[:-1])
    dlast = np.where(np.isnan(data.triton_field))[0][0]
    b = data.triton_field[:dlast]
    arrayn35_830_300[i, :] = np.interp(barr, b, data.lockin830_X[:dlast])
    arrayn35_865_300[i, :] = np.interp(barr, b, data.lockin865_X[:dlast])

In [180]:
np.savetxt('AnalysisNotebooks/W5_WAL_arrays/W5_10nA_-3p5Vbg_300mK_40-39.txt', arrayn35_865_300, fmt='%.8e', header='#array of lockin865 voltage readings (divide by 10e-9 to get Rxx) at 300mK for electrodes 40-39, with axis 0 being backgate voltage from -3.26 (top) to -3.76 (bottom) every 0.02 V (26 pts) and field from -25 to 25 mT, every 0.2 mT (251 pts)')
np.savetxt('AnalysisNotebooks/W5_WAL_arrays/W5_10nA_-3p5Vbg_300mK_39-38.txt', arrayn35_830_300, fmt='%.8e', header='#array of lockin830 voltage readings (divide by 10e-9 to get Rxx) at 300mK for electrodes 39-38, with axis 0 being backgate voltage from -3.26 (top) to -3.76 (bottom) every 0.02 V (26 pts) and field from -25 to 25 mT, every 0.2 mT (251 pts)')

500 mK

In [181]:
files_n35_500 = sorted(glob.glob('data/2020-03-29/#*/'))[58:84]

In [182]:
arrayn35_830_500 = np.zeros((ng_n35, nB))
arrayn35_865_500 = np.zeros((ng_n35, nB))

for i, f in enumerate(files_n35_500):
    data = qc.load_data(f[:-1])
    dlast = np.where(np.isnan(data.triton_field))[0][0]
    b = data.triton_field[:dlast]
    arrayn35_830_500[i, :] = np.interp(barr, b, data.lockin830_X[:dlast])
    arrayn35_865_500[i, :] = np.interp(barr, b, data.lockin865_X[:dlast])

In [183]:
np.savetxt('AnalysisNotebooks/W5_WAL_arrays/W5_10nA_-3p5Vbg_500mK_40-39.txt', arrayn35_865_500, fmt='%.8e', header='#array of lockin865 voltage readings (divide by 10e-9 to get Rxx) at 500mK for electrodes 40-39, with axis 0 being backgate voltage from -3.26 (top) to -3.76 (bottom) every 0.02 V (26 pts) and field from -25 to 25 mT, every 0.2 mT (251 pts)')
np.savetxt('AnalysisNotebooks/W5_WAL_arrays/W5_10nA_-3p5Vbg_500mK_39-38.txt', arrayn35_830_500, fmt='%.8e', header='#array of lockin830 voltage readings (divide by 10e-9 to get Rxx) at 500mK for electrodes 39-38, with axis 0 being backgate voltage from -3.26 (top) to -3.76 (bottom) every 0.02 V (26 pts) and field from -25 to 25 mT, every 0.2 mT (251 pts)')

700 mK

In [218]:
files_n35_700 = sorted(glob.glob('data/2020-03-30/#*/'))[18:44]

In [219]:
arrayn35_830_700 = np.zeros((ng_n35, nB))
arrayn35_865_700 = np.zeros((ng_n35, nB))

for i, f in enumerate(files_n35_700):
    data = qc.load_data(f[:-1])
    dlast = np.where(np.isnan(data.triton_field))[0][0]
    b = data.triton_field[:dlast]
    arrayn35_830_700[i, :] = np.interp(barr, b, data.lockin830_X[:dlast])
    arrayn35_865_700[i, :] = np.interp(barr, b, data.lockin865_X[:dlast])

In [220]:
np.savetxt('AnalysisNotebooks/W5_WAL_arrays/W5_10nA_-3p5Vbg_700mK_40-39.txt', arrayn35_865_700, fmt='%.8e', header='#array of lockin865 voltage readings (divide by 10e-9 to get Rxx) at 700mK for electrodes 40-39, with axis 0 being backgate voltage from -3.26 (top) to -3.76 (bottom) every 0.02 V (26 pts) and field from -25 to 25 mT, every 0.2 mT (251 pts)')
np.savetxt('AnalysisNotebooks/W5_WAL_arrays/W5_10nA_-3p5Vbg_700mK_39-38.txt', arrayn35_830_700, fmt='%.8e', header='#array of lockin830 voltage readings (divide by 10e-9 to get Rxx) at 700mK for electrodes 39-38, with axis 0 being backgate voltage from -3.26 (top) to -3.76 (bottom) every 0.02 V (26 pts) and field from -25 to 25 mT, every 0.2 mT (251 pts)')

900 mK

In [276]:
files_n35_900 = sorted(glob.glob('data/2020-03-30/#*/'))[70:96]

In [277]:
arrayn35_830_900 = np.zeros((ng_n35, nB))
arrayn35_865_900 = np.zeros((ng_n35, nB))

for i, f in enumerate(files_n35_900):
    data = qc.load_data(f[:-1])
    dlast = np.where(np.isnan(data.triton_field))[0][0]
    b = data.triton_field[:dlast]
    arrayn35_830_900[i, :] = np.interp(barr, b, data.lockin830_X[:dlast])
    arrayn35_865_900[i, :] = np.interp(barr, b, data.lockin865_X[:dlast])

In [278]:
np.savetxt('AnalysisNotebooks/W5_WAL_arrays/W5_10nA_-3p5Vbg_900mK_40-39.txt', arrayn35_865_900, fmt='%.8e', header='#array of lockin865 voltage readings (divide by 10e-9 to get Rxx) at 900mK for electrodes 40-39, with axis 0 being backgate voltage from -3.26 (top) to -3.76 (bottom) every 0.02 V (26 pts) and field from -25 to 25 mT, every 0.2 mT (251 pts)')
np.savetxt('AnalysisNotebooks/W5_WAL_arrays/W5_10nA_-3p5Vbg_900mK_39-38.txt', arrayn35_830_900, fmt='%.8e', header='#array of lockin830 voltage readings (divide by 10e-9 to get Rxx) at 900mK for electrodes 39-38, with axis 0 being backgate voltage from -3.26 (top) to -3.76 (bottom) every 0.02 V (26 pts) and field from -25 to 25 mT, every 0.2 mT (251 pts)')

# Near -6

In [238]:
files_n6_base = sorted(glob.glob('data/2020-03-27/#*/'))[71:97]
files_n6_base

['data/2020-03-27/#072_W5_865_40-39_830_39-38_baseT_WALfieldsw_Vbg-5.74_10nA_13-42-33/',
 'data/2020-03-27/#073_W5_865_40-39_830_39-38_baseT_WALfieldsw_Vbg-5.76_10nA_13-53-40/',
 'data/2020-03-27/#074_W5_865_40-39_830_39-38_baseT_WALfieldsw_Vbg-5.78_10nA_14-04-44/',
 'data/2020-03-27/#075_W5_865_40-39_830_39-38_baseT_WALfieldsw_Vbg-5.80_10nA_14-15-48/',
 'data/2020-03-27/#076_W5_865_40-39_830_39-38_baseT_WALfieldsw_Vbg-5.82_10nA_14-26-52/',
 'data/2020-03-27/#077_W5_865_40-39_830_39-38_baseT_WALfieldsw_Vbg-5.84_10nA_14-37-59/',
 'data/2020-03-27/#078_W5_865_40-39_830_39-38_baseT_WALfieldsw_Vbg-5.86_10nA_14-49-08/',
 'data/2020-03-27/#079_W5_865_40-39_830_39-38_baseT_WALfieldsw_Vbg-5.88_10nA_15-00-15/',
 'data/2020-03-27/#080_W5_865_40-39_830_39-38_baseT_WALfieldsw_Vbg-5.90_10nA_15-11-20/',
 'data/2020-03-27/#081_W5_865_40-39_830_39-38_baseT_WALfieldsw_Vbg-5.92_10nA_15-22-27/',
 'data/2020-03-27/#082_W5_865_40-39_830_39-38_baseT_WALfieldsw_Vbg-5.94_10nA_15-33-34/',
 'data/2020-03-27/#08

In [239]:
nB = 251
barr = np.linspace(-0.025, 0.025, nB)  # we will trim this down later when we combine arrays.
garr_n6 = np.linspace(-5.74, -6.24, 26)
ng_n6 = len(garr_n6)
arrayn6_830_base = np.zeros((ng_n6, nB))
arrayn6_865_base = np.zeros((ng_n6, nB))

for i, f in enumerate(files_n6_base):
    data = qc.load_data(f[:-1])
    dlast = np.where(np.isnan(data.triton_field))[0][0]
    b = data.triton_field[:dlast]
    arrayn6_830_base[i, :] = np.interp(barr, b, data.lockin830_X[:dlast])
    arrayn6_865_base[i, :] = np.interp(barr, b, data.lockin865_X[:dlast])

In [240]:
arrayn6_830_base.shape

(26, 251)

In [241]:
np.savetxt('AnalysisNotebooks/W5_WAL_arrays/W5_10nA_-6Vbg_baseT_40-39.txt', arrayn6_865_base, fmt='%.8e', header='#array of lockin865 voltage readings (divide by 10e-9 to get Rxx) at base Temp for electrodes 40-39, with axis 0 being backgate voltage from -5.74 (top) to -6.24 (bottom) every 0.02 V (26 pts) and field from -25 to 25 mT, every 0.2 mT (251 pts)')
np.savetxt('AnalysisNotebooks/W5_WAL_arrays/W5_10nA_-6Vbg_baseT_39-38.txt', arrayn6_830_base, fmt='%.8e', header='#array of lockin830 voltage readings (divide by 10e-9 to get Rxx) at base Temp for electrodes 39-38, with axis 0 being backgate voltage from -5.74 (top) to -6.24 (bottom) every 0.02 V (26 pts) and field from -25 to 25 mT, every 0.2 mT (251 pts)')

In [242]:
dGn6_865_base = np.zeros(arrayn6_865_base.shape)
pind = npd.val_to_index([-1.19/1000], barr)[0]
for i in range(arrayn6_865_base.shape[0]):
    dGn6_865_base[i, :] = 1/(arrayn6_865_base[i, :]/10e-9)/esqh - 1/(arrayn6_865_base[i, pind]/10e-9)/esqh

plt.figure()  # lockin865 (40-39)
# plt.pcolormesh(barr*1000, garr_n35, gaussian_filter((1/(arrayn35_830/10e-9)/esqh - 29.5), (1, 1)))
plt.pcolormesh(barr*1000, garr_n6, gaussian_filter(dGn6_865_base, (0.8, 0.8)))
plt.colorbar()
plt.ylabel('V$_{g}$')
plt.xlabel('B (mT)')
# plt.savefig(figfile + 'W5_near-6WALpeak_2d_40-39_raw.png', dpi=300, bbox_inches='tight')

-0.00 is not a value in the array
Plotted -0.00 instead


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'B (mT)')

In [925]:
a = plt.axis()

In [926]:
plt.figure()  # lockin865 (40-39),
plt.plot(barr*1000, 1/np.average(arrayn6_865[:]/10e-9, axis=0)/esqh - 48.73, '.r')
plt.axis(a)
plt.xlabel('B (mT)')
plt.ylabel('$\sigma(B) - \sigma(0)$ ($e^2/h$)')
# plt.savefig(figfile + 'W5_near-6WALpeak_avg8pts_40-39_raw.png', dpi=300, bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [244]:
dGn6_830_base = np.zeros(arrayn6_830_base.shape)
pind = npd.val_to_index([-1.19/1000], barr)[0]
for i in range(arrayn6_830_base.shape[0]):
    dGn6_830_base[i, :] = 1/(arrayn6_830_base[i, :]/10e-9)/esqh - 1/(arrayn6_830_base[i, pind]/10e-9)/esqh

plt.figure()  # lockin830 (39-38)
# plt.pcolormesh(barr*1000, garr_n35, gaussian_filter((1/(arrayn35_830/10e-9)/esqh - 29.5), (1, 1)))
plt.pcolormesh(barr*1000, garr_n6, gaussian_filter(dGn6_830_base, (0.8, 0.8)))
plt.colorbar()
plt.ylabel('V$_{g}$')
plt.xlabel('B (mT)')
# plt.savefig(figfile + 'W5_near-6WALpeak_2d_39-38_raw.png', dpi=300, bbox_inches='tight')

-0.00 is not a value in the array
Plotted -0.00 instead


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'B (mT)')

In [918]:
plt.figure()  # lockin830 (39-38),
plt.plot(barr*1000, 1/np.average(arrayn6_830[4:]/10e-9, axis=0)/esqh - 73.5, '.r')
plt.axis(a)
plt.xlabel('B (mT)')
plt.ylabel('$\sigma(B) - \sigma(0)$ ($e^2/h$)')
# plt.savefig(figfile + 'W5_near-6WALpeak_avg8pts_39-38_raw.png', dpi=300, bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [916]:
a = plt.axis()

300 mK

In [245]:
files_n6_300 = sorted(glob.glob('data/2020-03-29/#*/'))[2:28]

In [246]:
arrayn6_830_300 = np.zeros((ng_n6, nB))
arrayn6_865_300 = np.zeros((ng_n6, nB))

for i, f in enumerate(files_n6_300):
    data = qc.load_data(f[:-1])
    dlast = np.where(np.isnan(data.triton_field))[0][0]
    b = data.triton_field[:dlast]
    arrayn6_830_300[i, :] = np.interp(barr, b, data.lockin830_X[:dlast])
    arrayn6_865_300[i, :] = np.interp(barr, b, data.lockin865_X[:dlast])

In [247]:
np.savetxt('AnalysisNotebooks/W5_WAL_arrays/W5_10nA_-6Vbg_300mK_40-39.txt', arrayn6_865_300, fmt='%.8e', header='#array of lockin865 voltage readings (divide by 10e-9 to get Rxx) at 300 mK for electrodes 40-39, with axis 0 being backgate voltage from -5.74 (top) to -6.24 (bottom) every 0.02 V (26 pts) and field from -25 to 25 mT, every 0.2 mT (251 pts)')
np.savetxt('AnalysisNotebooks/W5_WAL_arrays/W5_10nA_-6Vbg_300mK_39-38.txt', arrayn6_830_300, fmt='%.8e', header='#array of lockin830 voltage readings (divide by 10e-9 to get Rxx) at 300 mK for electrodes 39-38, with axis 0 being backgate voltage from -5.74 (top) to -6.24 (bottom) every 0.02 V (26 pts) and field from -25 to 25 mT, every 0.2 mT (251 pts)')

500 mK

In [259]:
files_n6_500 = sorted(glob.glob('data/2020-03-29/#*/'))[84:110]

In [260]:
arrayn6_830_500 = np.zeros((ng_n6, nB))
arrayn6_865_500 = np.zeros((ng_n6, nB))

for i, f in enumerate(files_n6_500):
    data = qc.load_data(f[:-1])
    dlast = np.where(np.isnan(data.triton_field))[0][0]
    b = data.triton_field[:dlast]
    arrayn6_830_500[i, :] = np.interp(barr, b, data.lockin830_X[:dlast])
    arrayn6_865_500[i, :] = np.interp(barr, b, data.lockin865_X[:dlast])

In [261]:
np.savetxt('AnalysisNotebooks/W5_WAL_arrays/W5_10nA_-6Vbg_500mK_40-39.txt', arrayn6_865_500, fmt='%.8e', header='#array of lockin865 voltage readings (divide by 10e-9 to get Rxx) at 500 mK for electrodes 40-39, with axis 0 being backgate voltage from -5.74 (top) to -6.24 (bottom) every 0.02 V (26 pts) and field from -25 to 25 mT, every 0.2 mT (251 pts)')
np.savetxt('AnalysisNotebooks/W5_WAL_arrays/W5_10nA_-6Vbg_500mK_39-38.txt', arrayn6_830_500, fmt='%.8e', header='#array of lockin830 voltage readings (divide by 10e-9 to get Rxx) at 500 mK for electrodes 39-38, with axis 0 being backgate voltage from -5.74 (top) to -6.24 (bottom) every 0.02 V (26 pts) and field from -25 to 25 mT, every 0.2 mT (251 pts)')

700 mK

In [265]:
files_n6_700 = sorted(glob.glob('data/2020-03-30/#*/'))[44:70]

In [266]:
arrayn6_830_700 = np.zeros((ng_n6, nB))
arrayn6_865_700 = np.zeros((ng_n6, nB))

for i, f in enumerate(files_n6_700):
    data = qc.load_data(f[:-1])
    dlast = np.where(np.isnan(data.triton_field))[0][0]
    b = data.triton_field[:dlast]
    arrayn6_830_700[i, :] = np.interp(barr, b, data.lockin830_X[:dlast])
    arrayn6_865_700[i, :] = np.interp(barr, b, data.lockin865_X[:dlast])

In [267]:
np.savetxt('AnalysisNotebooks/W5_WAL_arrays/W5_10nA_-6Vbg_700mK_40-39.txt', arrayn6_865_700, fmt='%.8e', header='#array of lockin865 voltage readings (divide by 10e-9 to get Rxx) at 700 mK for electrodes 40-39, with axis 0 being backgate voltage from -5.74 (top) to -6.24 (bottom) every 0.02 V (26 pts) and field from -25 to 25 mT, every 0.2 mT (251 pts)')
np.savetxt('AnalysisNotebooks/W5_WAL_arrays/W5_10nA_-6Vbg_700mK_39-38.txt', arrayn6_830_700, fmt='%.8e', header='#array of lockin830 voltage readings (divide by 10e-9 to get Rxx) at 700 mK for electrodes 39-38, with axis 0 being backgate voltage from -5.74 (top) to -6.24 (bottom) every 0.02 V (26 pts) and field from -25 to 25 mT, every 0.2 mT (251 pts)')

900 mK

In [282]:
files_n6_900 = sorted(glob.glob('data/2020-03-30/#*/'))[96:122]

In [283]:
arrayn6_830_900 = np.zeros((ng_n6, nB))
arrayn6_865_900 = np.zeros((ng_n6, nB))

for i, f in enumerate(files_n6_900):
    data = qc.load_data(f[:-1])
    dlast = np.where(np.isnan(data.triton_field))[0][0]
    b = data.triton_field[:dlast]
    arrayn6_830_900[i, :] = np.interp(barr, b, data.lockin830_X[:dlast])
    arrayn6_865_900[i, :] = np.interp(barr, b, data.lockin865_X[:dlast])

In [284]:
np.savetxt('AnalysisNotebooks/W5_WAL_arrays/W5_10nA_-6Vbg_900mK_40-39.txt', arrayn6_865_900, fmt='%.8e', header='#array of lockin865 voltage readings (divide by 10e-9 to get Rxx) at 900 mK for electrodes 40-39, with axis 0 being backgate voltage from -5.74 (top) to -6.24 (bottom) every 0.02 V (26 pts) and field from -25 to 25 mT, every 0.2 mT (251 pts)')
np.savetxt('AnalysisNotebooks/W5_WAL_arrays/W5_10nA_-6Vbg_900mK_39-38.txt', arrayn6_830_900, fmt='%.8e', header='#array of lockin830 voltage readings (divide by 10e-9 to get Rxx) at 900 mK for electrodes 39-38, with axis 0 being backgate voltage from -5.74 (top) to -6.24 (bottom) every 0.02 V (26 pts) and field from -25 to 25 mT, every 0.2 mT (251 pts)')